In [146]:
from abc import abstractmethod

import numpy as np
import pandas as pd

from chimera import Chimera

import olympus
from olympus.objects import (
    ParameterContinuous,
    ParameterDiscrete,
    ParameterCategorical,
    abstract_attribute, 
    ABCMeta,
    Object
)
from olympus import Logger
from olympus.campaigns import Campaign, ParameterSpace
from olympus.surfaces import Surface
from olympus.planners import Planner



In [125]:
# get some random data from some objective functions

num_samples = 5

surface = Surface(kind='MultFonseca')

planner = Planner(kind='RandomSearch')
planner.set_param_space(surface.param_space)

[WARNING] Surface MultFonseca is only defined with 2 objectives: setting `value_dim`=2


In [126]:
params = []
values = []
for _ in range(num_samples):
    p = planner.ask()
    m = surface.run(p.to_array())[0][0]
    print(p.to_array())
    params.append(p.to_array())
    values.append(m)
    
params = np.array(params)
values = np.array(values)

[0.51376931 0.16997366]
[0.27998735 0.17088086]
[0.16193189 0.15986085]
[0.71475943 0.88012594]
[0.43459608 0.62676236]


In [127]:
params.shape, values.shape

((5, 2), (5, 2))

In [128]:
class AbstractASF(Object, metaclass=ABCMeta):
    
    def __init__(self, *args, **kwargs):
        Object.__init__(self, *args, **kwargs)
        
    
    @abstractmethod
    def scalarize(self, objectives):
        
        return None
    
    @abstractmethod
    def validate_asf_params(self):
        
        return None

## Chimera

In [129]:
class ChimeraASF(AbstractASF):
    ''' The Chimera achievement scalarizing function. 
    Chem. Sci., 2018, 9, 7642
    '''
    def __init__(self, value_space, tolerances, absolutes, goals):
        AbstractASF.__init__(**locals())
        
        self.validate_asf_params()
        
        self.chimera = Chimera(
            tolerances=self.tolerances, absolutes=self.absolutes, goals=self.goals
        )
        
        
    def scalarize(self, objectives):
        ''' this expects a (# obs, # objs) numpy array, which is scalarized 
        according to the given tolerances and goals. Returns a (# obs,) 
        numpy array corresponding to the merits of each observation, 0 corresponding
        to the best value, and 1 corresponding to the worst value
        '''
        assert len(objectives.shape)==2
        
        return self.chimera.scalarize(objectives)
    
    def validate_asf_params(self):
        
        if not (len(self.tolerances)==len(self.absolutes)==len(self.goals)):
            print('lengths of chimera params dont match')
        if not len(self.tolerances)==len(self.value_space):
            print('number of chimera params does not match the number of objectives')
        
        return None
        
    

In [130]:
tolerances = [0.4, 0.2]
absolutes= [False, False]
goals = ['min', 'min']

asf = ChimeraASF(surface.value_space, tolerances, absolutes, goals)

In [131]:
values

array([[0.2781164 , 0.89562953],
       [0.37497851, 0.82539293],
       [0.44937087, 0.77839551],
       [0.02954883, 0.98933738],
       [0.07754557, 0.95416431]])

In [132]:
asf.scalarize(values)

array([0.44378958, 0.75838409, 1.        , 0.18188625, 0.        ])

## Weighted sum

In [142]:
class WeightedSumASF(AbstractASF):
    ''' simple weighted sum acheivement scalarizing function
    weights is a 1d numpy array of 
    '''
    def __init__(self, value_space, weights, goals):
        AbstractASF.__init__(**locals())
        
        self.validate_asf_params()
        # normalize the weight values such that their magnitudes 
        # sum to 1
        self.norm_weights = self.softmax(self.weights)
        self.norm_weights = [weight if self.goals[idx]=='min' else -weight for idx, weight in enumerate(self.norm_weights)]
        
    def scalarize(self, objectives):
        norm_objectives = self.normalize(objectives)
        merit = np.sum(norm_objectives*self.norm_weights, axis=1)
        # final normalization
        # smaller merit values are best
        merit = self.normalize(merit)
    
        return merit
    
    @staticmethod
    def softmax(vector):
        vector = vector/np.amax(weights)
        
        return np.exp(weights) / np.sum(np.exp(weights))
    
    @staticmethod
    def normalize(vector):
        min_ = np.amin(vector)
        max_ = np.amax(vector)
        ixs = np.where(np.abs(max_-min_)<1e-10)[0]
        if not ixs.size == 0:
            max_[ixs]=np.ones_like(ixs)
            min_[ixs]=np.zeros_like(ixs)
        return (vector - min_) / (max_ - min_)
    
    def validate_asf_params(self):
        if not np.all(np.array(self.weights)>=0.):
            print('weights must be non-negative real numbers')
        if not len(self.weights)==len(self.value_space):
            print('number of weights does not match the number of objectives')
            

In [139]:
weights = [1., 1.]
goals   = ['max', 'max']

asf = WeightedSumASF(surface.value_space, weights, goals)

In [145]:
values
test_values = values[:, 0].reshape(-1, 1)
test_values

array([[0.2781164 ],
       [0.37497851],
       [0.44937087],
       [0.02954883],
       [0.07754557]])

In [144]:
asf.scalarize(values)

norm objectives :  [[0.25898159 0.90236615]
 [0.35990185 0.8291869 ]
 [0.43741097 0.78022048]
 [0.         1.        ]
 [0.05000762 0.96335332]]
norm weights :  [-0.5, -0.5]
merit :  [-0.58067387 -0.59454438 -0.60881572 -0.5        -0.50668047]
merit :  [0.25861933 0.13115148 0.         1.         0.9386075 ]


array([0.25861933, 0.13115148, 0.        , 1.        , 0.9386075 ])

## c-ASF

In [147]:
class ConstrainedASF(AbstractASF):
    
    def __init__(self, value_space, lowers, uppers, delta_fs):
        AbstractASF.__init__(**locals())
        
        self.validate_asf_params()
        

        
    def scalarize(self, objectives):
        return None
    
    def validate_asf_params(self):
        if not (len(self.lowers)==len(self.uppers)==len(self.delta_fs)):
            message = 'c-ASF parameters not the same length'
            Logger.log(message, 'FATAL')
        if not len(self.lowers) == len(self.value_space):
            message = 'Number of c-ASF parameters do not match the number of objectives'
            Logger.log(message, 'FATAL')
            
        